In [ ]:

import os.path as osp
import os
from glob import glob 
from rdkit import Chem
import pandas as pd
from PIL import Image
import io
from rdkit import DataStructs
from rdkit.Chem import Draw
from copy import deepcopy
from utils import compute_sims, generalize, find_match, sort_lists_by_first_list, imgs2singlePDF, read_sdf, write_sdf

def read_sdf(sdf_file, sanitize=False):
    supp = Chem.SDMolSupplier(sdf_file, sanitize=sanitize)
    mols_list = [i for i in supp]
    return mols_list

def write_sdf(mol_list,file, voice=False):
    writer = Chem.SDWriter(file)
    mol_cnt = 0
    for i in mol_list:
        try:
            writer.write(i)
            mol_cnt+=1
        except:
            pass
    writer.close()
    if voice: 
        print('Write {} molecules to {}'.format(mol_cnt,file))
def save_img(ipy_Image, out_file):
    img_byte_arr = io.BytesIO(ipy_Image.data)
    img_pil = Image.open(img_byte_arr)
    png_file = searched_file.replace('.sdf','.png')
    img_pil.save(out_file)
    print('saved at {}'.format(out_file))

In [ ]:
# seperate actives sdf to single active sdf 
kras_act = read_sdf('./KRAS_POSITIVE.sdf')
for i, mol in enumerate(kras_act):
    write_sdf([mol], f'./kras_active/{i}.sdf')

In [ ]:

all_mols = []
for i in range(8):
    sdf = f'./kras_active_sim_search/{i}_chemdiv_sim_100.sdf'
    mols = read_sdf(sdf)
    all_mols.extend(mols)
ids = [mol.GetProp('IDNUMBER') for mol in all_mols]
smiles = [mol.GetProp('Smile') for mol in all_mols]

info = pd.DataFrame({'Title': ids, 'Smiles': smiles})
info.to_csv('kras8act_ChemDiv_Sim_Search_Odin.csv')
write_sdf(all_mols, './kras8act_ChemDiv_Sim_Search.sdf')

In [51]:

all_mols = []
for i in range(8):
    sdf = f'./kras_active_sim_search/{i}_specs_sim_100.sdf'
    mols = read_sdf(sdf)
    all_mols.extend(mols)
ids = [mol.GetProp('IDNUMBER') for mol in all_mols]
smiles = [Chem.MolToSmiles(mol) for mol in all_mols]

info = pd.DataFrame({'Title': ids, 'Smiles': smiles})
info.to_csv('kras8act_Specs_Sim_Search_Odin.csv')
write_sdf(all_mols, './kras8act_Specs_Sim_Search.sdf')

In [ ]:

for i in range(8):
    sdf = f'./kras_active_sim_search/{i}_chemdiv_sim_100.sdf'
    pdf = f'./kras_active_sim_search_pdf/{i}_chemdiv_sim_100.pdf'
    mols = read_sdf(sdf)
    
    ids = [mol.GetProp('IDNUMBER') for mol in mols]
    number = list(range(len(mols)))
    legends = [f'No.{i[0]}; ID: {i[1]};' for i in zip(number, ids)]
    group_size = 52  
    show_mols_group = [mols[i:i + group_size] for i in range(0, len(mols), group_size)]
    legends_group = [legends[i:i + group_size] for i in range(0, len(legends), group_size)]
    imgs = []
    for show_mols, legend in zip(show_mols_group, legends_group):
        imgs.append(Draw.MolsToGridImage(show_mols, molsPerRow=4, subImgSize=(500,500), legends=legend,maxMols=group_size))
    imgs2singlePDF(imgs, pdf)

In [ ]:
ids = [mol.GetProp('IDNUMBER') for mol in all_mols]

In [ ]:
all_mols = []
for i in range(8):
    sdf = f'./kras_active_sim_search/{i}_specs_sim_100.sdf'
    mols = read_sdf(sdf)
    pdf = f'./kras_active_sim_search_pdf/{i}_specs_sim_100.pdf'
    ids = [mol.GetProp('IDNUMBER') for mol in mols]
    number = list(range(len(mols)))
    legends = [f'No.{i[0]}; ID: {i[1]};' for i in zip(number, ids)]
    group_size = 52  
    show_mols_group = [mols[i:i + group_size] for i in range(0, len(mols), group_size)]
    legends_group = [legends[i:i + group_size] for i in range(0, len(legends), group_size)]
    imgs = []
    for show_mols, legend in zip(show_mols_group, legends_group):
        imgs.append(Draw.MolsToGridImage(show_mols, molsPerRow=4, subImgSize=(500,500), legends=legend,maxMols=group_size))
    imgs2singlePDF(imgs, pdf)